In [17]:
import cv2
import torch
import numpy as np

from ultralytics.nn.tasks import DetectionModel

## Chargement du modèle .best

In [18]:
model_name = "y12_300_auto"

model_path = f"../../models/fine_tuned_models/{model_name}/weights/best.pt"

In [19]:
# Chargement du modèle sur le GPU 
checkpoint = torch.load(model_path, map_location=torch.device('cuda'))

model = checkpoint['model']
model.eval()

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(384, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   

## Définition des classes et du choix de la classe

In [20]:
classe_choisie = "fighter_white"

In [21]:
classes = [
    "fighter_white",
    "fighter_blue"
]

## Chargement et prétraitement de l'image

In [ ]:
image_path = "../../data/processed/dataset_fine_tuning_fighters/test_set/images/img_test2" \
".png"
image = cv2.imread(image_path)
if image is None:
    print("Erreur lors du chargement de l'image.")
    exit()

In [23]:
# réccupération de la taille de l'image initiale
height_img_init, width_img_init = image.shape[:2]

In [24]:
# Conversion de BGR vers RGB et redimensionnement (ici à 640x640, idem à modele entrainné)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = cv2.resize(image_rgb, (640, 640))

In [25]:
# Conversion en tensor, normalisation, ajout de la dimension batch,
# transfert sur GPU et conversion en half precision
input_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).float() / 255.0
input_tensor = input_tensor.unsqueeze(0)  # Ajout de la dimension batch
input_tensor = input_tensor.to(torch.device('cuda')).half()  # Passage en fp16

# Vérifier le type du tenseur avant l'inférence
print("Input tensor dtype:", input_tensor.dtype)  # Doit afficher torch.cuda.HalfTensor

Input tensor dtype: torch.float16


# Exécution de l'inférence

In [26]:
with torch.no_grad():
    predictions = model(input_tensor)

## Annotation de l'image et enregistrement

In [27]:
# Chargez l'image de test pour annotation
image_annotation = cv2.imread(image_path)
if image_annotation is None:
    raise ValueError("L'image n'a pas pu être chargée. Vérifiez le chemin.")


In [28]:
# Seuil de confiance pour considérer une prédiction comme valide
score_threshold = 0.35

In [29]:
# Réorganisation de la sortie :
# La prédiction initiale a la forme [1, 30, 8400]. On souhaite obtenir un tenseur de forme [8400, 30].
tensor_predictions = predictions[0]
preds = tensor_predictions.squeeze(0).transpose(0, 1) # shape : [8400, 30]

# predictions est un tenseur PyTorch, on le convertit en numpy pour faciliter le traitement avec OpenCV
preds = preds.cpu().detach().numpy()

In [30]:
# Iterration sur les prédictions
for pred in preds: 
    # Extraction des coordonnées pour la boîte englobante
    x1, y1, x2, y2 = pred[:4]

    # récupèration du score maximum parmi les classes et son indice
    class_scores = pred[4:]
    max_class_score = np.max(class_scores)
    class_idx = np.argmax(class_scores)
    
    # On vérifie que le score de la classe prédite est supérieur au seuil et uniquement pour la classe cherchée
    if max_class_score > score_threshold and classe_choisie == classes[class_idx] :
        
        # on réajuste à la taille de limage initiale
        # x1, y1 : coordonnées du centre de la boîte
        # x2, y2 : dimensions de la boîte
        x1 = (float(x1) * float(width_img_init) / 640.0)
        y1 = (float(y1) * float(height_img_init) / 640.0)
        x2 = (float(x2) * float(width_img_init) / 640.0)
        y2 = (float(y2) * float(height_img_init) / 640.0)

        # On defini les point d'encadrement de la boîte englobante
            # x1, y1 : coordonnées du centre de la boîte
            # x2, y2 : dimensions de la boîte
        start_point = (int(x1-(x2/2)), int(y1-(y2/2)))
        end_point   = (int(x1+(x2/2)), int(y1+(y2/2)))
        
        # Dessine le rectangle 
        cv2.rectangle(image_annotation, start_point, end_point, color=(102, 178, 255), thickness=2)
        
        # Prépare le texte à annoter (nom de la classe et score)
        label = f"{classes[class_idx]}: {max_class_score:.2f}"
        
        # Place le texte au-dessus de la boîte
        cv2.putText(image_annotation, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, (102, 178, 255), thickness=1)
        

In [31]:
# Enregistrement de l'image annotée
result_image_path = "/home/dim/clone_repo/BrickSearch/outputs/images_annotees/image_annotée.jpg"
cv2.imwrite(result_image_path, image_annotation)

True

In [32]:
# Affichage de l'image annotée

cv2.namedWindow("Image Annotée", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Image Annotée", 1024, 960)  # Redimensionne la fenêtre d'affichage
cv2.moveWindow("Image Annotée", 600, -40)     # Déplace la fenêtre d'affichage

# Affiche l'image annotée avec sa taille d'origine
image_annotation = cv2.resize(image_annotation, (width_img_init, height_img_init))
cv2.imshow("Image Annotée", image_annotation)

# Boucle pour attendre la fermeture de la fenêtre
while True:
    key = cv2.waitKey(100)
    # Si la fenêtre est fermée (valeur négative) ou que la touche 'q' est appuyée, on sort de la boucle
    if cv2.getWindowProperty("Image Annotée", cv2.WND_PROP_VISIBLE) < 1:
        break
    if key & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()